In [1]:
import pandas as pd
import numpy as np

## 데이터 전처리 (data_preprocessing.ipynb)
* Date Split
* Weekday
* Lunar Date
* Date Normalization
* 식사명 -> one-hot
* 식사내용 -> bag-of-wordMM

In [2]:
# Read Data
train_df = pd.read_excel("data/train.xlsx")
test_df = pd.read_excel("data/test.xlsx")

In [3]:
train_df.head()

,일자,식사명,식사내용,수량
0,20030301,아침,"과일샐러드,닭죽,돈육마늘장조림,떡만두국,부추김무침,쌀밥,딸기잼(중),비엔나구이,스크...",37.472924
1,20030301,저녁,"감자으깸샐러드,비프까스,스위트피클,쌀밥,옥수수스프",19.566787
2,20030301,점심(일반),"골뱅이야채무침,새우맛살튀김,쌀밥(사무직),열무겉절이,칼국수",31.191336
3,20030302,아침,"계란죽,곤약멸치조림,김치국,마카로니샐러드,쌀밥,오징어회무침,딸기잼(중),삶은계란,야...",36.101083
4,20030302,저녁,"계란탕,단무지잔파무침,자장소스,잡채밥,탕수만두",21.949458


In [4]:
test_df.head()

,일자,식사명,식사내용
0,20100713,아침,"누룽지,닭살찜닭소스조림,두유,멸치볶음,배추김치,쌀밥,열무된장무침,콩나물국,딸기잼(1..."
1,20100713,저녁,"배추김치,손만두국,쌀밥,애느타리볶음,오징어잔파무침,치커리사과생채"
2,20100713,점심(양식),"단무지,배추김치,쌀밥,야채샐러드,야채스프,치즈함박스테이크&데미"
3,20100713,점심(일반),"깍두기(손칼),돈등뼈감자탕,두부달걀전,쌀밥,쫄면야채무침,토마토화채,풋고추된장무침"
4,20100714,아침,"배추김치,쇠고기가지볶음,시금치된장국,쌀밥,야채죽,진미도라지무침,팽이계란전,화인쿨,딸..."


In [5]:
# df = pd.concat([train_df, test_df])
df = train_df.sort_values(by=['일자']).reset_index(drop=True)
print(df[:3])
print(df[-3:])

         일자     식사명                                               식사내용  \
0  20030301      아침  과일샐러드,닭죽,돈육마늘장조림,떡만두국,부추김무침,쌀밥,딸기잼(중),비엔나구이,스크...   
1  20030301      저녁                        감자으깸샐러드,비프까스,스위트피클,쌀밥,옥수수스프   
2  20030301  점심(일반)                   골뱅이야채무침,새우맛살튀김,쌀밥(사무직),열무겉절이,칼국수   

          수량  
0  37.472924  
1  19.566787  
2  31.191336  
             일자     식사명                                               식사내용  \
20007  20171231  점심(양식)            깍두기(손칼),식빵&딸기잼,쌀밥,야채샐러드,옥수수스프,피클,함박스테이크   
20008  20171231      아침  김가루잔파무침,누룽지,두유,배추김치,식빵&딸기잼,쌀밥,야채샐러드,오징어두루치기,탕국...   
20009  20171231  점심(일반)      계란파국,깍두기(손칼),날치알김치덮밥,생선까스&타르,쌀밥,오이생채,케이준치킨샐러드   

              수량  
20007   3.610108  
20008  22.166065  
20009   8.158845  


In [6]:
w_feature = ['평균기온(°C)','최저기온(°C)','최고기온(°C)','최대 풍속(m/s)','평균 풍속(m/s)','평균 이슬점온도(°C)',
             '평균 상대습도(%)','가조시간(hr)','합계 일조 시간(hr)','평균 지면온도(°C)','합계 일사(MJ/m2)']

def add_weather(df):
    weather = pd.read_excel("data/[과제3] 관련. 포항지역 날씨 데이터(03~17).xlsx")
    weather['일시'] = [int(x.strftime("%Y%m%d")) for x in weather['일시']]
    weather = weather[w_feature+['일시', '기사']]
    
    gisa_onehot = []
    str_gisa = weather['기사'].astype(str)
    check_w = ['비', '박무', '연무', '채운', '눈', '황사', '소나기', 
               '진눈깨비', '천둥', '뇌전', '번개', '우박', '햇무리', '달무리']
    w_col = ["기사_"+x for x in check_w]
    for s in str_gisa:
        temp_w = []
        for c in check_w:
            if c in s:
                temp_w.append(1)
            else:
                temp_w.append(0)
        gisa_onehot.append(temp_w)
    gisa_onehot = np.array(gisa_onehot)
    weather = weather.join(pd.DataFrame(gisa_onehot, columns=w_col))
    weather.drop(['기사'], axis=1, inplace=True)
    
    for x in weather.columns:
        if weather[x].isnull().sum() < len(weather):
            if x not in check_w:
                weather[x].interpolate(inplace=True)
            if x != '일시':
                weather[x] = weather[x].shift(3)
    
    df = pd.merge(df, weather, how='left', left_on='일자', right_on='일시')
    df.drop(['일시'], axis=1, inplace=True)
    
    return df

# 식사명 변환 (one-hot)
def convert_ont_hot(df):
    df = df.join(pd.get_dummies(df['식사명'], prefix='식사명'))
    df.drop(['식사명'], axis=1, inplace=True)
    return df

def moving_average(df):
    def ma_window(df, window_size):
        morning = df['수량'][df['식사명_아침']==1].rolling(window_size, min_periods=1).mean().shift(3)
        lunch = df['수량'][df['식사명_점심(일반)']==1].rolling(window_size, min_periods=1).mean().shift(3)
        lunch_west = df['수량'][df['식사명_점심(양식)']==1].rolling(window_size, min_periods=1).mean().shift(3)
        dinner = df['수량'][df['식사명_저녁']==1].rolling(window_size, min_periods=1).mean().shift(3)    
        return pd.concat([morning, lunch, lunch_west, dinner]).sort_index()
#     df['MA_1day'] = ma_window(df, 1)
#     df['MA_3day'] = ma_window(df, 3)
#     df['MA_5day'] = ma_window(df, 5)
    df['MA_week'] = ma_window(df, 7)/100
    df['MA_month'] = ma_window(df, 30)/100
    df['MA_half_year'] = ma_window(df, 180)/100
    df['MA_year'] = ma_window(df, 365)/100
    df.drop(df[df.일자 < 20040326].index, inplace=True)
    df.drop(df[(df.일자 > 20050109) & (df.일자 < 20060331) & (df['식사명_점심(양식)']==1)].index, inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

# 식사내용 변환 (Bag-of-Word)
import konlpy
import re
def menu_dictionary(menu_list):
    def tokenize(text):
        text = text.replace('.', ',')
        text = text.replace('/', ',')
        text = text.replace('&', ',')
        return text.split(',')
    menu = []
    for m in menu_list:
        for mm in tokenize(m):
            mm = re.sub('\(.*?\)', '', mm)
            mm = re.sub('[0-9a-zA-Z]', '', mm)
            menu.append(mm)
    menu = list(set(menu))
    
    okt = konlpy.tag.Okt()
    menu_morphs = []
    for m in menu:
        for ok in okt.morphs(m):
            if len(ok) > 1:
                menu_morphs.append(ok)
    menu_morphs += ['떡', '국', '회', '탕', '밥', '닭', '죽', '찜']
    menu_morphs = list(set(menu_morphs))
    return menu_morphs

def convert_bow(df):
    menu = menu_dictionary(df['식사내용'])
    bow =[]
    for row in df['식사내용']:
        vec = []
        for m in menu:
            if m in row:
                vec.append(1)
            else:
                vec.append(0)
        bow.append(vec)
    bow = np.array(bow)
    df = df.join(pd.DataFrame(bow, columns=menu))
    df.drop(['식사내용'], axis=1, inplace=True)
    return df

# 년/월/일 분리(split) + 요일(Weekday) 추가
def split_date(df):
    # Normalize Date
    df['year'] = (df['일자'] / 10000).astype(int)
    df['month'] = (df['일자'] % 10000 / 100).astype(int)
    df['day'] = (df['일자'] % 100).astype(int)
    df['weekday'] = pd.to_datetime(df['일자'], format = '%Y%m%d').dt.dayofweek
    return df

# 음력 추가
from korean_lunar_calendar import KoreanLunarCalendar
from datetime import datetime

def add_lunar_date(df):
    calendar = KoreanLunarCalendar()
    
    lunar_y = []
    lunar_m = []
    lunar_d = []
    for y, m, d in zip (df['year'], df['month'], df['day']):
        calendar.setSolarDate(y, m, d)
        lunar_date = calendar.LunarIsoFormat()
        lunar_y.append(int(lunar_date[:4]))
        lunar_m.append(int(lunar_date[5:7]))
        lunar_d.append(int(lunar_date[8:10]))
        
    df['lunar_year'], df['lunar_month'], df['lunar_day'] = lunar_y, lunar_m, lunar_d
    return df

# 년/월/일 변환
def year_norm(df):
    df['year'] = (df['year']-min(df['year'])) / (max(df['year'])-min(df['year']))
    df['lunar_year'] = (df['lunar_year']-min(df['lunar_year'])) / (max(df['lunar_year'])-min(df['lunar_year']))
    return df
def month_norm(df):
    df['month_sin'] = [np.sin(x*2*np.pi/12) for x in df['month']]
    df['month_cos'] = [np.cos(x*2*np.pi/12) for x in df['month']]
    df['lunar_month_sin'] = [np.sin(x*2*np.pi/12) for x in df['lunar_month']]
    df['lunar_month_cos'] = [np.cos(x*2*np.pi/12) for x in df['lunar_month']]
    df.drop(['month', 'lunar_month'], axis=1, inplace=True)
    return df
def day_norm(df):
    df['day_sin'] = [np.sin(x*2*np.pi/31) for x in df['day']]
    df['day_cos'] = [np.cos(x*2*np.pi/31) for x in df['day']]
    df['lunar_ay_sin'] = [np.sin(x*2*np.pi/31) for x in df['lunar_day']]
    df['lunar_day_cos'] = [np.cos(x*2*np.pi/31) for x in df['lunar_day']]
    df.drop(['day', 'lunar_day'], axis=1, inplace=True)
    return df
def weekday_norm(df):
    df['weekday_sin'] = [np.sin(x*2*np.pi/7) for x in df['weekday']]
    df['weekday_cos'] = [np.cos(x*2*np.pi/7) for x in df['weekday']]
    return df

In [7]:
import time

In [8]:
start = time.time()
# add Weather Information
df = add_weather(df)
print("add weather feature done..{}s".format(time.time() - start))

start = time.time()
# convert 식사명 to one-hot
df = convert_ont_hot(df)
print("convert one-hot about title done..{}s".format(time.time() - start))

start = time.time()
# Moving Average of 수량
df = moving_average(df)
print("moving average done..{}s".format(time.time() - start))


start = time.time()
# convert 식사내용 to Bag-of-Word Vector
df = convert_bow(df)
print("bag of word vector on contents done..{}s".format(time.time() - start))


start = time.time()
# Date
df = split_date(df)
print("split date done..{}s".format(time.time() - start))

start = time.time()
df = add_lunar_date(df)
print("add lunar date done..{}s".format(time.time() - start))


start = time.time()
# Date Normalization
df = year_norm(df)
df = month_norm(df)
df = day_norm(df)
df = weekday_norm(df)
print("date normalization done..{}s".format(time.time() - start))

add weather feature done..3.278904914855957s
convert one-hot about title done..0.01081991195678711s
moving average done..0.05610799789428711s
bag of word vector on contents done..10.743458986282349s
split date done..0.01683211326599121s
add lunar date done..89.4630970954895s
date normalization done..0.68361496925354s


In [9]:
lunch = []
for a, b in zip(df['식사명_점심(일반)'], df['식사명_점심(양식)']):
    if a==1 or b==1:
        lunch.append(1)
    else:
        lunch.append(0)
df['식사명_점심'] = lunch

In [10]:
date = sorted(list(set(df['일자'])))

one_lunch = []
for d in date:
    if df[df['일자']==d]['식사명_점심(양식)'].sum() == 0:
        for _ in range(len(df[df['일자']==d])):
            one_lunch.append(1)
    else:
        for _ in range(len(df[df['일자']==d])):
            one_lunch.append(0)
df['점심하나줌'] = one_lunch

In [11]:
print("Number of Columns =", len(df.columns))
df.head()

Number of Columns = 851


,일자,수량,평균기온(°C),최저기온(°C),최고기온(°C),최대 풍속(m/s),평균 풍속(m/s),평균 이슬점온도(°C),평균 상대습도(%),가조시간(hr),...,lunar_month_sin,lunar_month_cos,day_sin,day_cos,lunar_ay_sin,lunar_day_cos,weekday_sin,weekday_cos,식사명_점심,점심하나줌
0,20040326,41.877256,10.5,4.6,17.6,6.3,3.0,3.5,62.5,12.3,...,0.866025,0.5,-0.848644,0.528964,0.937752,0.347305,-0.433884,-0.900969,0,1
1,20040326,13.718412,10.5,4.6,17.6,6.3,3.0,3.5,62.5,12.3,...,0.866025,0.5,-0.848644,0.528964,0.937752,0.347305,-0.433884,-0.900969,0,1
2,20040326,40.361011,10.5,4.6,17.6,6.3,3.0,3.5,62.5,12.3,...,0.866025,0.5,-0.848644,0.528964,0.937752,0.347305,-0.433884,-0.900969,1,1
3,20040327,37.184116,11.5,7.2,15.6,4.6,2.5,-0.3,46.0,12.3,...,0.866025,0.5,-0.724793,0.688967,0.988468,0.151428,-0.974928,-0.222521,0,1
4,20040327,12.635379,11.5,7.2,15.6,4.6,2.5,-0.3,46.0,12.3,...,0.866025,0.5,-0.724793,0.688967,0.988468,0.151428,-0.974928,-0.222521,0,1


In [12]:
df.head()

,일자,수량,평균기온(°C),최저기온(°C),최고기온(°C),최대 풍속(m/s),평균 풍속(m/s),평균 이슬점온도(°C),평균 상대습도(%),가조시간(hr),...,lunar_month_sin,lunar_month_cos,day_sin,day_cos,lunar_ay_sin,lunar_day_cos,weekday_sin,weekday_cos,식사명_점심,점심하나줌
0,20040326,41.877256,10.5,4.6,17.6,6.3,3.0,3.5,62.5,12.3,...,0.866025,0.5,-0.848644,0.528964,0.937752,0.347305,-0.433884,-0.900969,0,1
1,20040326,13.718412,10.5,4.6,17.6,6.3,3.0,3.5,62.5,12.3,...,0.866025,0.5,-0.848644,0.528964,0.937752,0.347305,-0.433884,-0.900969,0,1
2,20040326,40.361011,10.5,4.6,17.6,6.3,3.0,3.5,62.5,12.3,...,0.866025,0.5,-0.848644,0.528964,0.937752,0.347305,-0.433884,-0.900969,1,1
3,20040327,37.184116,11.5,7.2,15.6,4.6,2.5,-0.3,46.0,12.3,...,0.866025,0.5,-0.724793,0.688967,0.988468,0.151428,-0.974928,-0.222521,0,1
4,20040327,12.635379,11.5,7.2,15.6,4.6,2.5,-0.3,46.0,12.3,...,0.866025,0.5,-0.724793,0.688967,0.988468,0.151428,-0.974928,-0.222521,0,1


In [13]:
df.isnull().any()

일자                 False
수량                 False
평균기온(°C)           False
최저기온(°C)           False
최고기온(°C)           False
최대 풍속(m/s)         False
평균 풍속(m/s)         False
평균 이슬점온도(°C)       False
평균 상대습도(%)         False
가조시간(hr)           False
합계 일조 시간(hr)       False
평균 지면온도(°C)        False
합계 일사(MJ/m2)       False
기사_비               False
기사_박무              False
기사_연무              False
기사_채운              False
기사_눈               False
기사_황사              False
기사_소나기             False
기사_진눈깨비            False
기사_천둥              False
기사_뇌전              False
기사_번개              False
기사_우박              False
기사_햇무리             False
기사_달무리             False
식사명_아침             False
식사명_저녁             False
식사명_점심(양식)         False
                   ...  
쫄면                 False
생채                 False
돈까츠동               False
국물                 False
닭고기                False
수재                 False
메추리                False
부대찌개               False
개국                 False


In [18]:
def train_and_predict(model, train_x, train_y, dev_x):
    for w in w_feature:
        mean = train_x[w].mean()
        std = train_x[w].std()
        train_x[w] = (train_x[w] - mean) / std
        dev_x[w] = (dev_x[w] - mean) / std
    model.fit(train_x, train_y)
    return model.predict(dev_x)

In [21]:
def target_oneline(model, df, target_date):
    df_datelist = df['일자'].unique()
    if target_date not in df_datelist:
        print("no target date in datelist.. select another one")
        return
    _target_df = df[df['일자'] == target_date]
    _target_gt = _target_df['수량']
    _target_x = _target_df.drop(['수량'], axis=1)
    # print(_target_x)
    _train_df = df[df['일자'] < target_date - 2]
    _train_y = _train_df['수량']
    _train_x = _train_df.drop(['수량'], axis=1)
    pred = train_and_predict(model, 
                             _train_x,
                             _train_y,
                             _target_x)
    # print("pred : {}".format(pred))
    return pred, _target_gt
    

In [16]:
import xgboost as xgb
model = xgb.XGBRegressor(n_estimators=300, 
                         learning_rate=0.05, 
                         max_depth=7,
                         colsample_bytree=0.9,
                         gamma=0.4,
                         subsample=0.8,
                         reg_alpha=0.01,
                         seed=10,
                         n_jobs=-1)

In [22]:
target_date = 20150303
pred, gt = target_oneline(model, df, target_date)
print(pred)
gt

[28.57598  23.29166  20.423164 25.392405]


14653    27.148014
14654    24.332130
14655    25.270758
14656    21.877256
Name: 수량, dtype: float64